# Desarrollo

## Actividad 1

## Preguntas:
* [a](#a)
* [b](#b)
* [c](#c)
* [d](#d)
* [e](#e)
* [f](#f)
* [g](#g)
* [h](#h)
* [i](#i)
* [j](#j)
* [k](#k)
* [l](#l)
* [m](#m)
* [n](#n)
* [o](#o)
* [p](#p)
* [q](#p)
* [r](#p)
* [s](#s)
* [t](#t)
* [u](#u)
* [v](#v)
* [Actividad 2](#Actividad2)

In [1]:
import pandas as pd
ftr = open("train_data.csv", "r",  encoding="ISO-8859-1")
rows = [line.split(" ",1) for line in ftr.readlines()]
df_train = pd.DataFrame(rows, columns=['Sentiment','Text'])
df_train['Sentiment'] = (pd.to_numeric(df_train['Sentiment'])+1)/2 # 0 o 1

fts = open("test_data.csv", "r",  encoding="ISO-8859-1")
rows = [line.split(" ",1) for line in fts.readlines()]
df_test = pd.DataFrame(rows, columns=['Sentiment','Text'])
df_test['Sentiment'] = (pd.to_numeric(df_test['Sentiment'])+1)/2 # 0 o 1

df_train_text = df_train.Text
df_test_text = df_test.Text
labels_train = df_train.Sentiment.values
labels_test = df_test.Sentiment.values

a)<a class="anchor" id="a"></a>

In [2]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3554 entries, 0 to 3553
Data columns (total 2 columns):
Sentiment    3554 non-null float64
Text         3554 non-null object
dtypes: float64(1), object(1)
memory usage: 55.7+ KB


In [3]:
df_train.head()

,Sentiment,Text
0,0.0,"everything's serious , poetic , earnest and --..."
1,0.0,"narratively , trouble every day is a plodding ..."
2,1.0,a truly wonderful tale combined with stunning ...
3,1.0,jason patric and ray liotta make for one splen...
4,0.0,haneke keeps us at arm's length . guided more ...


In [4]:
suma = 0
i = 0
for sentence in df_train['Text']:
    if i == 0:
        minimo = len(sentence)
        maximo = len(sentence)
    else:
        if minimo>len(sentence):
            minimo = len(sentence)
            min_sent = sentence
        if maximo<len(sentence):
            maximo = len(sentence)
            max_sent = sentence
    i+=1
    suma += len(sentence)
avg_len = suma/len(df_train['Text'])

In [5]:
(avg_len, minimo, maximo, min_sent)

(114.70709060213844, 7, 267, 'crummy\n')

In [6]:
df_test.head()

,Sentiment,Text
0,0.0,screenwriter dan schneider and director shawn ...
1,0.0,"home alone goes hollywood , a funny premise un..."
2,1.0,seldom has a movie so closely matched the spir...
3,0.0,"less dizzying than just dizzy , the jaunt is p..."
4,0.0,an ultra-low-budget indie debut that smacks mo...


In [7]:
suma = 0
i = 0
for sentence in df_test['Text']:
    if i == 0:
        minimo = len(sentence)
        maximo = len(sentence)
    else:
        if minimo>len(sentence):
            minimo = len(sentence)
            min_sent = sentence
        if maximo<len(sentence):
            maximo = len(sentence)
            max_sent = sentence
    i+=1
    suma += len(sentence)
avg_len = suma/len(df_train['Text'])

In [8]:
(avg_len, minimo, maximo, min_sent)

(116.4679234665166, 9, 268, 'horrible\n')

Para un mejor analisis preliminar seria de utilidad la eliminacion de las stopwords a la hora de poder determinar los largos maximo, minimo y el promedio del dataset

b)<a class="anchor" id="b"></a>

In [9]:
from sklearn.model_selection import train_test_split
df_train_text, df_val_text, labels_train, labels_val  = train_test_split(df_train, labels_train, test_size=0.2, random_state=0)

In [10]:
df_train_text.shape

(2843, 2)

In [11]:
df_val_text.shape

(711, 2)

c)<a class="anchor" id="c"></a>

In [12]:
import re, time
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer, word_tokenize
def base_word(word):
    wordlemmatizer = WordNetLemmatizer()
    return wordlemmatizer.lemmatize(word) 
def word_extractor(text):
    commonwords = stopwords.words('english')
    text = re.sub(r'([a-z])\1+', r'\1\1',text) #substitute multiple letter by two
    words = ""
    wordtokens = [ base_word(word.lower()) for word in word_tokenize(text) ]
    for word in wordtokens:
        if word not in commonwords: #delete stopwords
            words+=" "+word
    return words
... #try yourself
word_extractor("I love to eat cake")
word_extractor("I love eating cake")
word_extractor("I loved eating the cake")
word_extractor("I do not love eating cake")
word_extractor("I don't love eating cake")
... #try yourself
texts_train = [word_extractor(text[1]) for text in df_train_text.to_numpy()]
texts_val = [word_extractor(text[1]) for text in df_val_text.to_numpy()]
texts_test = [word_extractor(text[1]) for text in df_test.to_numpy()]

La importancia del preprocesamiento en el dominio del lenguaje natural radica en disminuir las palabras que si bien aparecen con mas frecuencia son las menos significantes para realizar analisis (las stopwords) para asi reducir ruido y tiempo de ejecucion. Tambien es importante el proceso de reducir las palabras a su origen con stemming o lematizacion para evitar analizar palabras cuya semantica es identica pero estan escritas diferentes producto de una conjugacion. En este caso se utiliza lematizacion ya que presenta mejores resultados que stemming al realizar el proceso considerando el contexto en el cual se encuentra la palabra.

d)<a class="anchor" id="d"></a>

In [13]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 1), binary=False) #TF representation
vectorizer.fit(texts_train)
features_train = vectorizer.transform(texts_train)
... #transform val and test


vectorizer2 = CountVectorizer(ngram_range=(1, 1), binary=False)
vectorizer2.fit(texts_val)
features_val = vectorizer.transform(texts_val)

vectorizer3 = CountVectorizer(ngram_range=(1, 1), binary=False)
vectorizer3.fit(texts_test)
features_test = vectorizer.transform(texts_test)

vocab = vectorizer.get_feature_names()
dist=list(np.array(features_train.sum(axis=0)).reshape(-1,))

vocab2 = vectorizer.get_feature_names()
dist2=list(np.array(features_val.sum(axis=0)).reshape(-1,))

vocab3 = vectorizer.get_feature_names()
dist3=list(np.array(features_test.sum(axis=0)).reshape(-1,))

e)<a class="anchor" id="e"></a>

In [14]:
sentiment = []
for s in labels_train:
    if s == 1:
        sentiment.append("Positivo")
    else:
        sentiment.append("Negativo")

In [15]:
from sklearn.decomposition import TruncatedSVD #aka LSA
import matplotlib.pyplot as plt
model = TruncatedSVD(n_components=2)
model.fit(features_train)
x_plot = model.transform(features_train)
plt.figure(figsize=(10,5))
plt.scatter(x_plot[:,0], x_plot[:,1], c=labels_train)
plt.show()

<Figure size 1000x500 with 1 Axes>

In [16]:
np.where(x_plot == x_plot.max())

(array([1786], dtype=int64), array([0], dtype=int64))

In [17]:
x_plot[1786]

array([1.95015737, 1.75218186])

In [18]:
sentiment[1786]

'Positivo'

Amarillo Positivo, Morado Negativo

Las agrupaciones que se forman parecen contener una cantidad equitativa de positivo y negativo. Estas agrupaciones corresponden a las tematicas de las distintas criticas, pudiendo ser que cada agrupacion sea una pelicula distinta o que hablen sobre caracteristicas de las peliculas como la musica, los actores, etc.

f)<a class="anchor" id="f"></a>

In [25]:
from sklearn.linear_model import LogisticRegression
from ipywidgets import interact, interactive, fixed, interact_manual

from sklearn.metrics import confusion_matrix
import seaborn as sn
%matplotlib inline

def do_LOGIT(x,y,xv,yv, param):
    print("Param C= ",param)
    model= LogisticRegression(penalty = 'l2')
    model.set_params(C=param)
    model.fit(x,y)
    train_acc = model.score(x,y)
    test_acc = model.score(xv,yv)
    return model, train_acc, test_acc
#Cs = [10**i for i in np.arange(-4,4)]
Cs = np.power(np.repeat(10., 8),np.arange(-4,4))
def interact_doLOGIT(Css):
    model, train_acc, test_acc = do_LOGIT(features_train,labels_train,features_val,labels_val, param= Css )
    print("train acc: ",train_acc)
    print("test acc: ",test_acc)
    
    #Train
    lista_train = model.predict(features_train)
    #
    y_true_train = list(map(str,lista_train))
    y_pred_train = list(map(str,labels_train))
    data = confusion_matrix(y_true_train, y_pred_train)
    df_cm = pd.DataFrame(data, columns=np.unique(y_true_train), index = np.unique(y_true_train))
    df_cm.index.name = 'Actual'
    df_cm.columns.name = 'Predicted Train'
    plt.figure(figsize = (10,7))
    sn.set(font_scale=1.2)#for label size
    sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 14})# font size
    
    #Val
    lista_val = model.predict(features_val)
    #
    y_true_val = list(map(str,lista_val))
    y_pred_val = list(map(str,labels_val))
    data = confusion_matrix(y_true_val, y_pred_val)
    df_cm = pd.DataFrame(data, columns=np.unique(y_true_val), index = np.unique(y_true_val))
    df_cm.index.name = 'Actual'
    df_cm.columns.name = 'Predicted Val'
    plt.figure(figsize = (10,7))
    sn.set(font_scale=1.2)#for label size
    sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 14})# font size
    
interact(interact_doLOGIT, Css=Cs)

    

interactive(children=(Dropdown(description='Css', options=(0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0)…

<function __main__.interact_doLOGIT(Css)>

Conclusiones de lo que se observa de los heatmaps

g)<a class="anchor" id="g"></a>